In [1]:
from util import *
import time
import datetime

import cPickle as pickle
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import pylab
import re
import scipy as sp
import seaborn
import sklearn.feature_extraction.text as text

from gensim import corpora, models
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn import decomposition

In [2]:
lines=[]
with open("data/reviews_full.dat", "r") as fh:
    lines = fh.readlines()  

In [3]:
userid = []
rating = []
docs = []
business = []
i = 0
j = 0
error_line_num = []
error_lines = []
for line in lines:
    try:
        i = i + 1
        l = line.split('\t', 4)
        userid.append(l[0])
        business.append(l[1])
        rating.append(l[2])
        docs.append(l[3])
        #d = clean(l[3])
        #kmers = getKmers(d)
        #d.extend(kmers)
        #docs.append(d)
    except Exception as e:
        j = j + 1
        error_line_num.append(i)
        error_lines.append(line)

print 'Training Data: Number of lines processed: ' + str(i)
print 'Training Data: Length of userid array: ' + str(len(userid))
print 'Training Data: Length of rating array: ' + str(len(rating))
print 'Training Data: Length of docs array: ' + str(len(docs))
print 'Training Data: Length of business array: ' + str(len(business))
print 'Training Data: Number of exceptions encountered: ' + str(j)



Training Data: Number of lines processed: 317586
Training Data: Length of userid array: 317586
Training Data: Length of rating array: 317586
Training Data: Length of docs array: 317586
Training Data: Length of business array: 317586
Training Data: Number of exceptions encountered: 0


In [ ]:

from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
n_samples = 50000
n_features = 15000
n_components = 30
n_top_words = 50

data_samples = docs[:n_samples]

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))


# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)

tfidf_nmf = nmf.transform(tfidf)

print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
#tfidf_feature_names = tfidf_vectorizer.get_feature_names()
vocab = np.array(tfidf_vectorizer.get_feature_names())
#print_top_words(nmf, tfidf_feature_names, n_top_words)




Extracting tf-idf features for NMF...


In [6]:
trainTopics= tfidf_nmf
trainTopics = trainTopics / np.sum(trainTopics, axis=1, keepdims=True)


In [7]:
print(trainTopics)

[[  3.59808943e-02   9.59386431e-12   1.05263099e-18 ...,   1.74009988e-06
    9.08278472e-02   5.77900043e-05]
 [  2.21546670e-02   1.07138315e-09   1.62203576e-17 ...,   9.20884092e-03
    9.39471413e-02   2.17619282e-02]
 [  1.09653566e-01   2.19063583e-09   3.42787903e-14 ...,   1.88932181e-01
    5.56722344e-02   2.69888782e-02]
 ..., 
 [  3.07061546e-01   3.10969950e-06   1.88197927e-13 ...,   9.21558359e-08
    7.15075417e-02   4.65283049e-02]
 [  2.47713001e-02   6.10084190e-14   5.27626498e-12 ...,   6.41727361e-08
    1.06995187e-02   8.72246562e-02]
 [  2.67701033e-04   2.45861637e-09   6.13062077e-05 ...,   3.00691072e-05
    3.16014715e-05   1.27476527e-01]]


In [8]:
d, f = trainTopics.shape
cols = ["Topic"+str(i) for i in xrange(1, f+1)]
nmfDF = pd.DataFrame(trainTopics, columns=cols)

In [9]:
nmfDF['rating'] = map(float,rating[:50000])

In [10]:
nmfDF.T.head(25)

,0,1,2,3,4,5,6,7,8,9,...,49990,49991,49992,49993,49994,49995,49996,49997,49998,49999
Topic1,3.598089e-02,2.215467e-02,1.096536e-01,3.452927e-01,1.832872e-01,3.264535e-03,6.699393e-17,2.356691e-10,1.787541e-01,2.630244e-16,...,1.265918e-07,8.986759e-10,5.759451e-02,1.862052e-01,1.411870e-01,4.184803e-02,9.269040e-05,3.070615e-01,2.477130e-02,2.677010e-04
Topic2,9.593864e-12,1.071383e-09,2.190636e-09,4.537844e-09,1.180364e-34,9.857292e-02,4.509435e-06,1.859698e-05,9.966614e-02,2.584283e-19,...,4.263059e-28,0.000000e+00,4.836499e-05,4.819688e-13,1.878374e-01,6.467699e-36,2.607594e-12,3.109699e-06,6.100842e-14,2.458616e-09
Topic3,1.052631e-18,1.622036e-17,3.427879e-14,3.317914e-24,1.172674e-20,4.548222e-03,8.398694e-08,2.999145e-01,3.412576e-07,4.669176e-09,...,3.285503e-20,8.189673e-33,6.136910e-01,2.617178e-18,2.486470e-05,9.750550e-16,8.427065e-24,1.881979e-13,5.276265e-12,6.130621e-05
Topic4,6.722892e-02,3.296125e-01,7.688786e-05,1.421382e-09,5.359882e-11,3.756426e-04,1.149507e-13,4.041388e-33,3.721130e-05,1.161887e-06,...,7.333080e-21,8.840010e-02,4.321537e-07,3.103981e-08,1.015310e-07,3.940916e-01,1.141166e-17,1.544283e-16,5.498921e-01,7.122673e-09
Topic5,0.000000e+00,5.628071e-07,1.344935e-23,5.374052e-02,4.271617e-11,7.042254e-17,9.243502e-01,4.177864e-01,4.183456e-03,7.266795e-02,...,4.665090e-13,5.834792e-01,1.185617e-20,3.466843e-16,8.143483e-02,3.478479e-07,2.845355e-01,1.050856e-02,3.872887e-10,8.211567e-11
Topic6,3.148868e-13,1.213013e-06,3.976133e-09,1.050835e-09,5.453457e-08,1.518464e-01,1.897921e-19,4.750134e-20,5.548838e-12,1.874753e-04,...,4.709200e-14,1.215398e-24,4.497266e-09,4.295247e-02,2.883531e-02,2.619086e-23,6.689450e-08,8.947949e-02,1.582972e-16,1.751678e-01
Topic7,1.072947e-13,2.722556e-05,9.840823e-02,6.282900e-08,6.824645e-01,1.701781e-02,1.729117e-05,9.057283e-14,1.435410e-02,2.141666e-01,...,7.225820e-05,4.595956e-20,9.170527e-25,3.017087e-11,1.955223e-07,1.518099e-01,4.390795e-03,5.002173e-15,2.647459e-09,3.436517e-01
Topic8,6.339584e-02,2.361639e-09,7.743830e-03,7.061523e-09,2.538701e-12,2.993775e-13,8.993685e-14,3.766364e-19,2.864703e-03,1.616888e-16,...,4.724101e-21,2.739747e-01,8.485526e-03,1.516894e-06,1.233056e-02,1.329119e-10,2.474253e-01,2.070710e-06,3.215482e-08,6.807791e-14
Topic9,2.479650e-16,2.509525e-18,1.353406e-03,2.146298e-08,2.852465e-14,6.497585e-04,1.222254e-27,8.393151e-18,4.883968e-01,3.559268e-10,...,4.899841e-12,2.429595e-29,1.181709e-05,3.389155e-02,1.362419e-11,5.945294e-10,3.560487e-06,8.542437e-06,7.917810e-25,3.495692e-09
Topic10,6.107703e-14,2.996706e-02,1.165575e-02,2.315238e-01,8.398140e-06,1.211923e-01,0.000000e+00,3.439201e-07,4.255427e-02,2.413207e-01,...,6.046732e-10,1.357761e-20,3.624519e-08,3.531623e-01,1.026264e-01,2.447709e-08,3.957206e-20,1.656448e-05,2.865805e-02,7.087370e-04


In [11]:
def getSentiment(x):
    if x < 3.5:
        return 0
    else:
        return 1

In [12]:
nmfDF['Sentiment'] = nmfDF['rating'].map(getSentiment)

In [13]:
nmfDF = nmfDF.dropna(how='any')


In [14]:
nmfDF.T.head(25)

,0,1,2,3,4,5,6,7,8,9,...,49990,49991,49992,49993,49994,49995,49996,49997,49998,49999
Topic1,3.598089e-02,2.215467e-02,1.096536e-01,3.452927e-01,1.832872e-01,3.264535e-03,6.699393e-17,2.356691e-10,1.787541e-01,2.630244e-16,...,1.265918e-07,8.986759e-10,5.759451e-02,1.862052e-01,1.411870e-01,4.184803e-02,9.269040e-05,3.070615e-01,2.477130e-02,2.677010e-04
Topic2,9.593864e-12,1.071383e-09,2.190636e-09,4.537844e-09,1.180364e-34,9.857292e-02,4.509435e-06,1.859698e-05,9.966614e-02,2.584283e-19,...,4.263059e-28,0.000000e+00,4.836499e-05,4.819688e-13,1.878374e-01,6.467699e-36,2.607594e-12,3.109699e-06,6.100842e-14,2.458616e-09
Topic3,1.052631e-18,1.622036e-17,3.427879e-14,3.317914e-24,1.172674e-20,4.548222e-03,8.398694e-08,2.999145e-01,3.412576e-07,4.669176e-09,...,3.285503e-20,8.189673e-33,6.136910e-01,2.617178e-18,2.486470e-05,9.750550e-16,8.427065e-24,1.881979e-13,5.276265e-12,6.130621e-05
Topic4,6.722892e-02,3.296125e-01,7.688786e-05,1.421382e-09,5.359882e-11,3.756426e-04,1.149507e-13,4.041388e-33,3.721130e-05,1.161887e-06,...,7.333080e-21,8.840010e-02,4.321537e-07,3.103981e-08,1.015310e-07,3.940916e-01,1.141166e-17,1.544283e-16,5.498921e-01,7.122673e-09
Topic5,0.000000e+00,5.628071e-07,1.344935e-23,5.374052e-02,4.271617e-11,7.042254e-17,9.243502e-01,4.177864e-01,4.183456e-03,7.266795e-02,...,4.665090e-13,5.834792e-01,1.185617e-20,3.466843e-16,8.143483e-02,3.478479e-07,2.845355e-01,1.050856e-02,3.872887e-10,8.211567e-11
Topic6,3.148868e-13,1.213013e-06,3.976133e-09,1.050835e-09,5.453457e-08,1.518464e-01,1.897921e-19,4.750134e-20,5.548838e-12,1.874753e-04,...,4.709200e-14,1.215398e-24,4.497266e-09,4.295247e-02,2.883531e-02,2.619086e-23,6.689450e-08,8.947949e-02,1.582972e-16,1.751678e-01
Topic7,1.072947e-13,2.722556e-05,9.840823e-02,6.282900e-08,6.824645e-01,1.701781e-02,1.729117e-05,9.057283e-14,1.435410e-02,2.141666e-01,...,7.225820e-05,4.595956e-20,9.170527e-25,3.017087e-11,1.955223e-07,1.518099e-01,4.390795e-03,5.002173e-15,2.647459e-09,3.436517e-01
Topic8,6.339584e-02,2.361639e-09,7.743830e-03,7.061523e-09,2.538701e-12,2.993775e-13,8.993685e-14,3.766364e-19,2.864703e-03,1.616888e-16,...,4.724101e-21,2.739747e-01,8.485526e-03,1.516894e-06,1.233056e-02,1.329119e-10,2.474253e-01,2.070710e-06,3.215482e-08,6.807791e-14
Topic9,2.479650e-16,2.509525e-18,1.353406e-03,2.146298e-08,2.852465e-14,6.497585e-04,1.222254e-27,8.393151e-18,4.883968e-01,3.559268e-10,...,4.899841e-12,2.429595e-29,1.181709e-05,3.389155e-02,1.362419e-11,5.945294e-10,3.560487e-06,8.542437e-06,7.917810e-25,3.495692e-09
Topic10,6.107703e-14,2.996706e-02,1.165575e-02,2.315238e-01,8.398140e-06,1.211923e-01,0.000000e+00,3.439201e-07,4.255427e-02,2.413207e-01,...,6.046732e-10,1.357761e-20,3.624519e-08,3.531623e-01,1.026264e-01,2.447709e-08,3.957206e-20,1.656448e-05,2.865805e-02,7.087370e-04


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.neighbors import KNeighborsRegressor

In [16]:
cols = [u'Topic1', u'Topic2', u'Topic3', u'Topic4', u'Topic5', u'Topic6', u'Topic7', u'Topic8', u'Topic9', u'Topic10', u'Topic11', u'Topic12', u'Topic13', u'Topic14', u'Topic15', u'Topic16', u'Topic17', u'Topic18', u'Topic19', u'Topic20', u'Sentiment']
Xtrain = nmfDF[:40000][cols]
Ytrain = nmfDF[:40000]['rating']
Xtest = nmfDF[40000:][cols]
Ytest = nmfDF[40000:]['rating']


#Xtrain_base = trainVectors[:18000]
#Ytrain_base = map(float,rating[:18000])
#Xtest_base = trainVectors[8000:10000]
#Ytest_base = map(float,rating[8000:10000])




In [17]:
from sklearn.metrics import mean_squared_error, r2_score
clfs = [ LogisticRegression(),KNeighborsRegressor(n_neighbors=3)]
clf_names = ['Logistic Regression','KNeighborsRegressor']
results = {}
for (i, clf_) in enumerate(clfs):
    clf = clf_.fit(Xtrain, Ytrain)
    preds = clf.predict(Xtest)
    
    #clf1 = clf_.fit(Xtrain_base, Ytrain_base)
    #preds1 = clf.predict(Xtest_base)
   
    
 
    #precision = metrics.precision_score(Ytest, preds)
    #recall = metrics.recall_score(Ytest, preds)
    #f1 = metrics.f1_score(Ytest, preds)
    #accuracy = accuracy_score(Ytest, preds)
    #report = classification_report(Ytest, preds)
    #matrix = metrics.confusion_matrix(Ytest, preds, labels=[1, 2, 3, 4, 5])
    score= r2_score(Ytest, preds)
    #score1= r2_score(Ytest_base, preds1)
    print "NMF: " +clf_names[i] + str(score)
    #print "Baseline : " +clf_names[i] + str(score)



NMF: Logistic Regression0.604207634956
NMF: KNeighborsRegressor0.671339846105


In [ ]:
print results